# Generate Dissimilar Groups

This notebook is to generate groups that optimize the amount of time every student spends in groups with different students. The basic rules are to specifcy any groups to avoid and to create groups that don't have more than two of the same students as before.

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import random

In [2]:
# Read in students (if applicable) and convert to list

# Sample class of 16 students (generating four groups of four)
entire_class = ['Adam', 'Brenda', 'Chuck', 'Dora', 
                'Ernest', 'Fay', 'Guy', 'Helen', 
                'Igor', 'Julie', 'Kelly', 'Lisa',
                'Mike', 'Nancy', 'Oliver', 'Pat']

In [3]:
# Used when results are nested lists or tuples to flatten into a single list
def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

In [4]:
# Function to check if any group members are the same
def common_member(a, b): 
    return True if (set(a) & set(b)) else False

# Test it out
print(common_member([1, 2, 3], [6, 4, 5]))

False


In [5]:
# Function to check if more than 2 members are the same:
def similar_groups(group_1, group_2):
    same = 0
    for s1 in group_1:
        for s2 in group_2: 
            if s1 == s2:
                same +=1
    return True if same > 2 else False

# Test it out
print(f'Are the groups similar with 2 common members: {similar_groups([1, 2, 3, 4], [3, 4, 5, 6])}')
print(f'Are the groups similar with 3 common members: {similar_groups([1, 2, 3, 4], [2, 3, 4, 5])}')

Are the groups similar with 2 common members: False
Are the groups similar with 3 common members: True


In [6]:
# Generate all possible groups of four members
all_groups = list(combinations(entire_class, 4))
print(f'There are {len(all_groups)} possible groups.')

There are 1820 possible groups.


In [7]:
# Add any groups you don't want created:
past_groups = ['Adam', 'Guy', 'Oliver', 'Pat']

In [8]:
# Function to generate one set of dissimilar groups
def get_groups(all_groups = all_groups, past_groups = past_groups, n_groups = 4):
    groups = []
    for x in range(n_groups):
        next_group = random.choice(all_groups)
        while common_member(flatten(groups), next_group) or similar_groups(flatten(past_groups), next_group):
            next_group = random.choice(all_groups)
        groups.append(next_group)
    past_groups.append(groups)
    return groups

In [9]:
# Function to schedule a number of weeks of groups
def schedule_groups(past_groups = past_groups, weeks = 24):
    schedule = []
    for x in range(weeks):
        schedule.append(get_groups(past_groups = past_groups))
    return schedule

In [10]:
# Set the number of weeks (if not the default of 24)
weeks = 8

# Generate the groups as a DataFrame
group_schedule = pd.DataFrame(schedule_groups(weeks = weeks))
# Assign columns
group_schedule.columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
# Add a weeks column
group_schedule['week'] = range(1, weeks + 1)
# Set the index to the week
group_schedule = group_schedule.set_index('week')
group_schedule

,Group 1,Group 2,Group 3,Group 4
week,,,,
1,"(Brenda, Chuck, Nancy, Oliver)","(Fay, Igor, Lisa, Mike)","(Adam, Helen, Julie, Kelly)","(Dora, Ernest, Guy, Pat)"
2,"(Chuck, Ernest, Fay, Lisa)","(Adam, Dora, Igor, Nancy)","(Brenda, Julie, Kelly, Pat)","(Guy, Helen, Mike, Oliver)"
3,"(Brenda, Igor, Mike, Oliver)","(Adam, Chuck, Fay, Lisa)","(Dora, Guy, Nancy, Pat)","(Ernest, Helen, Julie, Kelly)"
4,"(Dora, Kelly, Mike, Nancy)","(Fay, Guy, Oliver, Pat)","(Adam, Brenda, Helen, Igor)","(Chuck, Ernest, Julie, Lisa)"
5,"(Fay, Helen, Nancy, Pat)","(Adam, Chuck, Guy, Mike)","(Brenda, Igor, Julie, Oliver)","(Dora, Ernest, Kelly, Lisa)"
6,"(Dora, Guy, Mike, Nancy)","(Fay, Julie, Kelly, Oliver)","(Chuck, Helen, Lisa, Pat)","(Adam, Brenda, Ernest, Igor)"
7,"(Brenda, Igor, Kelly, Oliver)","(Fay, Helen, Mike, Nancy)","(Adam, Guy, Lisa, Pat)","(Chuck, Dora, Ernest, Julie)"
8,"(Dora, Lisa, Mike, Nancy)","(Adam, Brenda, Helen, Kelly)","(Ernest, Guy, Igor, Pat)","(Chuck, Fay, Julie, Oliver)"


In [11]:
# Check a week
group_schedule.iloc[0]

Group 1    (Brenda, Chuck, Nancy, Oliver)
Group 2           (Fay, Igor, Lisa, Mike)
Group 3       (Adam, Helen, Julie, Kelly)
Group 4          (Dora, Ernest, Guy, Pat)
Name: 1, dtype: object

In [12]:
# Export to csv
group_schedule.to_csv('groups.csv')